In [ ]:
from pathlib import Path
import datetime
import traceback

import awkward as ak
import dask
import dask_awkward as dak
import hist.dask
import coffea
import numpy as np
import uproot
from dask.distributed import Client, performance_report
import matplotlib.pyplot as plt

from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import PackedSelection
from coffea import dataset_tools

from functools import partial
import os
import time
import warnings

import utils  # worker count tracking

executor = "dask"   # "dask" or "taskvine" or "dask_gateway"

warnings.filterwarnings("ignore")
NanoAODSchema.warn_missing_crossrefs = False # silences warnings about branches we will not use here

    
print(f"awkward: {ak.__version__}")
print(f"dask-awkward: {dak.__version__}")
print(f"uproot: {uproot.__version__}")
print(f"hist: {hist.__version__}")
print(f"coffea: {coffea.__version__}")

In [ ]:
scheduler_options = {}

# for coffea-casa
if executor == "taskvine":
    from ndcctools.taskvine import DaskVine, Task
    
    
    manager = DaskVine(port=8786, ssl=True, name=f"{os.environ.get('USER', 'noname')}-coffea-casa")

    extra_files = {}
    env_vars = {}
    
    token_acc_path = "/etc/cmsaf-secrets-chown/access_token"
    token_xch_path = "/etc/cmsaf-secrets-chown/xcache_token"

    if Path(token_acc_path).is_file():
        extra_files[manager.declare_file(token_acc_path, cache=True)] = "access_token"
        env_vars["BEARER_TOKEN_FILE"] = "access_token"
    if Path(token_xch_path).is_file():
        extra_files[manager.declare_file(token_xch_path, cache=True)] = "xcache_token"
        env_vars["XCACHE_FILE"] = "xcache_token"

    vine_scheduler = partial(manager.get,
                             resources={"cores": 1, "disk": 2000},  #  max 1 core, 5GB of disk per task
                             extra_files=extra_files,
                             env_vars=env_vars,
                             submit_per_cycle=1000,
                             lazy_transfers=True,
                             #  resources_mode=None,   # set to "fixed" to kill tasks on resources
                            )
    # change default scheduler
    scheduler_options['scheduler'] = vine_scheduler
elif executor == "dask_gateway":
    num_workers = 200   #number of workers desired
    from dask.distributed import LocalCluster, Client, progress
    gateway = Gateway()
    clusters=gateway.list_clusters()
    cluster = gateway.connect(clusters[0].name)
    client = cluster.get_client()
    cluster.scale(num_workers)
    # %%
    def set_env(dask_worker):
        path = str(pathlib.Path(dask_worker.local_directory) / 'access_token')
        os.environ["BEARER_TOKEN_FILE"] = path
        os.chmod(path, 0o600)

    def try_xrdcp():
        import subprocess
        subprocess.run(["xrdcp", "-f", "root://xcache//store/user/IDAP/zstd_files.json", "zstd_files.json"])
    
    client.wait_for_workers(num_workers)
    client.upload_file("/etc/cmsaf-secrets/access_token")
    client.run(set_env)
        
else:
    # by default use dask   
    # local: single thread, single worker
    from dask.distributed import LocalCluster, Client, progress
    
    # cluster = LocalCluster(n_workers=1, processes=False, threads_per_worker=1)
    # client = Client(cluster)
    client = Client("tls://localhost:8786")

In [ ]:
import json
fname = "zstd_files.json"
fileset = {}
with open(fname,'r') as fp:
    for i,(dataset_name,file_list) in enumerate(json.load(fp).items()):
        fileset[dataset_name] = {"files": {}}
        for j,dataset_fpath in enumerate(file_list):
            xrd_fpath = f"root://xcache/{dataset_fpath}"
            fileset[dataset_name]["files"][xrd_fpath] = "Events"

In [ ]:
# apply optional filtering to limit number of input files

# limite to the first N files per container, None if no limit
LIMIT_NUM_FILES = None

# limit to the first N containers, None if no limit
LIMIT_NUM_CONTAINERS = None

fileset = coffea.dataset_tools.max_files(fileset, LIMIT_NUM_FILES)

if LIMIT_NUM_CONTAINERS is not None:
    fileset = dict((k,v) for i, (k,v) in enumerate(fileset.items()) if i <LIMIT_NUM_CONTAINERS)

print(f"number of input files after filter: {sum([len(f['files']) for f in fileset.values()])}")

In [ ]:
# turn fileset into simple list of files to run over
all_files = []
for process in fileset:
    all_files += fileset[process]["files"]

# define work to be done
def uproot_open_materialize(fname):
    BRANCH_LIST = [
        "GenPart_pt", "GenPart_eta", "GenPart_phi", "CorrT1METJet_phi",
        "GenJet_pt", "CorrT1METJet_eta", "SoftActivityJet_pt",
        "Jet_eta", "Jet_phi", "SoftActivityJet_eta", "SoftActivityJet_phi", 
        "CorrT1METJet_rawPt", "Jet_btagDeepFlavB", "GenJet_eta", 
        "GenPart_mass", "GenJet_phi",
        "Jet_puIdDisc", "CorrT1METJet_muonSubtrFactor", "Jet_btagDeepFlavCvL",
        "Jet_btagDeepFlavQG", "Jet_mass", "Jet_pt", "GenPart_pdgId",
        "Jet_btagDeepFlavCvB", "Jet_cRegCorr"
        ]

    filter_name = lambda x: x in BRANCH_LIST

    size_uncompressed = 0
    t0 = time.perf_counter()
    try:
        with uproot.open(fname, filter_name=filter_name) as f:
            num_entries = f["Events"].num_entries
            for b in BRANCH_LIST:
                f["Events"][b].array()
                size_uncompressed += f["Events"][b].uncompressed_bytes

            size_read = f.file.source.num_requested_bytes
        exception = None

    except:
        num_entries = 0
        size_read = 0
        size_uncompressed = 0
        exception = traceback.format_exc()

    t1 = time.perf_counter()
    time_finished = datetime.datetime.now()
    return {"fname": fname, "read": size_read, "uncompressed": size_uncompressed, "num_entries": num_entries,
            "runtime": t1-t0, "time_finished": time_finished, "exception": exception}

In [ ]:
# perform computation
print(f"running with {len(all_files)} files")
# scattered_data = client.scatter([f for f in all_files])  # instead of submitting (possibly big) object directly

utils.worker_tracking.start_tracking_workers(client)  # track worker count in background
t0 = time.perf_counter()
with performance_report(filename="dask-report-plain-uproot.html"):
    # futures = client.map(uproot_open_materialize, scattered_data)
    # out = ak.Array([r for r in client.gather(iter(futures))])
    tasks = [dask.delayed(uproot_open_materialize)(f) for f in all_files]
    out = ak.Array(dask.compute(*tasks))
t1 = time.perf_counter()
utils.worker_tracking.stop_tracking_workers()

print(f"wall clock time: {t1-t0:.2f}s")
print(f"current number of workers: {len(client.scheduler_info()['workers'])}")

timestamps, nworkers, avg_num_workers = utils.worker_tracking.get_timestamps_and_counts()  # worker count info

In [ ]:
# summary of performance
read_GB = sum(out['read']) / 1000**3
print(f"total read (compressed): {read_GB:.2f} GB")
print(f"total read (uncompressed): {sum(out['uncompressed']) / 1000**3:.2f} GB")

rate_Gbps = read_GB*8/(t1-t0)
print(f"average data rate: {rate_Gbps:.2f} Gbps (need to scale by x{200/rate_Gbps:.0f} to reach 200 Gbps)")

n_evts = sum(out["num_entries"])
print(f"total event rate (wall clock time): {n_evts / (t1-t0) / 1000:.2f} kHz (processed {n_evts} events total)")

total_runtime = sum(out["runtime"])
print(f"total aggregated runtime in function: {total_runtime:.2f} s")
print(f"ratio total runtime / wall clock time: {total_runtime / (t1-t0):.2f} "\
      "(should match # cores without overhead / scheduling issues)")
print(f"time-averaged number of workers: {avg_num_workers:.1f}")
print(f"\"efficiency\" (ratio of two numbers above): {total_runtime / (t1-t0) / avg_num_workers:.1%}")
print(f"event rate (aggregated time spent in function): {n_evts / total_runtime / 1000:.2f} kHz")

utils.worker_tracking.plot_worker_count(timestamps, nworkers, avg_num_workers)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def plot_worker_count(timestamps, nworkers, avg_num_workers, times_for_rates, instantaneous_rates):
    fig, ax1 = plt.subplots(constrained_layout=True)
    ax1.plot(timestamps, nworkers, linewidth=2, color="C0")
    ax1.set_xlabel("time")
    ax1.set_ylabel("number of workers", color="C0")
    ax1.set_ylim([0, ax1.get_ylim()[1]*1.1])
    # ax1.set_title(f"worker count over time, average: {avg_num_workers:.1f}")
    ax1.set_title(f"worker count and data rate over time")
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    ax1.tick_params(axis="y", labelcolor="C0")
                              
    for tick in ax1.get_xticklabels():
        tick.set_rotation(45)
        tick.set_horizontalalignment("right")
    
    ax2 = ax1.twinx()
    ax2.plot(times_for_rates, instantaneous_rates, marker="v", linewidth=0, color="C1")
    ax2.set_ylabel("data rate [Gbps]", color="C1")
    ax2.set_ylim([0, ax2.get_ylim()[1]*1.1])
    ax2.tick_params(axis="y", labelcolor="C1")
    fig.savefig("worker_count_data_rate_over_time.png")

In [ ]:
import matplotlib as mpl
mpl.style.use("ggplot")

# get arrays for starting time, runtime and end time of all tasks
runtimes = np.asarray([datetime.timedelta(seconds=t) for t in out["runtime"]], dtype=np.timedelta64)
ends = out["time_finished"].to_numpy()
starts = ends - runtimes

# calculate instantaneous rates for given timestamp
times_for_rates = []
instantaneous_rates = []
for t in timestamps[::10]:  # only calculate every 30 seconds
    mask = np.logical_and((starts <= t), (t <= ends))  # mask for tasks running at given timestamp
    rate_Gbps_at_timestamp = sum(out[mask]['read']*8 / 1000**3 / out[mask]["runtime"])
    # print(f"instantaneous rate at t={t}: {rate_Gbps_at_timestamp:.2f} Gbps")
    times_for_rates.append(t)
    instantaneous_rates.append(rate_Gbps_at_timestamp)

plot_worker_count(timestamps, nworkers, avg_num_workers, times_for_rates, instantaneous_rates)

In [ ]:
print(f"{sum(o is not None for o in out['exception'])} files failed\n")

# use below to get full list with details
# for report in out:
#     if report["exception"] is not None:
#         print(f"{report['fname']} failed in {report['runtime']:.2f} s\n{report['exception']}\n")

In [ ]:
fig, ax = plt.subplots()
bins = np.linspace(0, max(out["runtime"])*1.01, 100)
ax.hist(out["runtime"], bins=bins)
ax.set_xlabel("runtime [s]")
ax.set_xlim([0, ax.get_xlim()[1]])
ax.set_ylabel("count")
ax.semilogy();

In [ ]:
fig, ax = plt.subplots()
ax.scatter(out["num_entries"], out["runtime"], marker="x")
ax.set_xlabel("number of events")
ax.set_ylabel("runtime [s]")

# below: coffea 2024 approach

In [ ]:
def task(events):
    # track number of events
    num_events = ak.num(events, axis=0)

    # read out all other branches into integers to avoid memory issues
    _counter = 0
    for obj_to_add in [
        events.GenPart.pt,
        events.GenPart.eta,
        events.GenPart.phi,
        events.CorrT1METJet.phi,
        events.GenJet.pt, 
        events.CorrT1METJet.eta,
        events.SoftActivityJet.pt,
        events.Jet.eta,
        events.Jet.phi,
        events.SoftActivityJet.eta,
        events.SoftActivityJet.phi, 
        events.CorrT1METJet.rawPt,
        events.Jet.btagDeepFlavB,
        events.GenJet.eta, 
        events.GenPart.mass,
        events.GenJet.phi,
        events.Jet.puIdDisc,
        events.CorrT1METJet.muonSubtrFactor,
        events.Jet.btagDeepFlavCvL,
        events.Jet.btagDeepFlavQG,
        events.Jet.mass,
        events.Jet.pt,
        events.GenPart.pdgId,
        events.Jet.btagDeepFlavCvB,
        events.Jet.cRegCorr
        
    ]:
        _counter_to_add = ak.count_nonzero(obj_to_add, axis=1)

        # reduce >2-dimensional (per event) branches further
        for _ in range(_counter_to_add.ndim - 1):
            _counter_to_add = ak.count_nonzero(_counter_to_add, axis=-1)

        _counter = _counter + _counter_to_add  # sum 1-dim array built from new branch

    _counter = ak.count_nonzero(_counter, axis=0)  # reduce to int

    return {"nevts": num_events, "_counter": _counter}

In [ ]:
%%time
# pre-process
samples, report = dataset_tools.preprocess(fileset, skip_bad_files=True, uproot_options={"allow_read_errors_with_report": True}, **scheduler_options)

In [ ]:
# find issues where access did not work
for process in report:
    for k, v in report[process]["files"].items():
        if v["steps"] is None:
            print(f"could not read {k}")

In [ ]:
%%time
# create the task graph
tasks = dataset_tools.apply_to_fileset(task, samples, uproot_options={"allow_read_errors_with_report": (OSError, TypeError, KeyError)})

execute task graph

In [ ]:
%%time
# execute
utils.worker_tracking.start_tracking_workers(client)  # track worker count in background
t0 = time.perf_counter()

if executor == "taskvine":
    ((out, report),) = dask.compute(tasks, **scheduler_options)
else:
    with performance_report(filename="dask-report.html"):
        ((out, report),) = dask.compute(tasks, **scheduler_options)  # feels strange that this is a tuple-of-tuple
t1 = time.perf_counter()
utils.worker_tracking.stop_tracking_workers()

time_uproot = ak.sum([v['duration'] for v in report.values()])
print(f"total time spent in uproot reading data: {time_uproot:.2f} s")
print(f"wall time: {t1-t0:.2f}s")

timestamps, nworkers, avg_num_workers = utils.worker_tracking.get_timestamps_and_counts()  # worker count info

In [ ]:
print(f"output: {out}")

print("\nperformance metrics:")
event_rate = sum([out[process]["nevts"] for process in out.keys()]) / (t1-t0)
print(f" - event rate: {event_rate / 1_000:.2f} kHz")

# need uproot>=5.3.2 to get these useful performance stats
num_bytes = ak.sum([report[process]["performance_counters"]["num_requested_bytes"] for process in out.keys()])
read_MB = num_bytes / 1_000**2
rate_Mbs = read_MB / (t1-t0)
print(f" - read {read_MB:.2f} MB in {t1-t0:.2f} s -> {rate_Mbs*8:.2f} Mbps (need to scale by x{200/8/rate_Mbs*1000:.0f} to reach 200 Gbps)")
print(f" - time-averaged number of workers: {avg_num_workers:.1f}")
print(f" - spent {time_uproot:.1f} s reading data with wall time {t1-t0:.2f} and {avg_num_workers:.1f} cores on average -> \"efficiency\": {time_uproot / (t1-t0) / avg_num_workers:.1%}")

utils.worker_tracking.plot_worker_count(timestamps, nworkers, avg_num_workers)


In [ ]:
# report problematic files that caused exceptions
for process in report.keys():
    for i_file in range(len(report[process].exception)):
        file_report = report[process][i_file]
        if file_report.exception is not None:
            print(file_report.args[0].strip("\'"))
            print(file_report.message + "\n")